In [1]:
import sys
import os

from dotenv import load_dotenv

from agents import Agent

sys.path.append(os.path.join(os.path.dirname(os.getcwd()), "src"))

from mav.MAS.pipeline import Pipeline

from mav.benchmark import benchmark_suite_without_injections

from mav.Tasks.load_task_suites import get_suite
from mav.Tasks.utils._transform import convert_to_openai_function_tool

from mav.MAS.framework import MultiAgentSystem

load_dotenv()

True

In [ ]:
# loading a task suite
slack_task_suite = get_suite("slack")
banking_task_suite = get_suite("banking")

# convert to openai function tools
banking_openai_tools = []
slack_openai_tools = []

for tool in banking_task_suite.tools:
    try:
        banking_openai_tools.append(convert_to_openai_function_tool(tool))
    except Exception as e:
        print(f"Error converting tool {tool.__name__}: {e}")

for tool in slack_task_suite.tools:
    try:
        slack_openai_tools.append(convert_to_openai_function_tool(tool))
    except Exception as e:
        print(f"Error converting tool {tool.__name__}: {e}")

# create an multi-agent system
banking_agent = Agent(
    name="Banking Agent",
    instructions="""You are an intelligent banking agent that handles user queries using available tools.""",
    model="gpt-4o",
    tools=banking_openai_tools,
)

slack_agent = Agent(
    name="Slack Agent",
    instructions="""You are a Slack agent that interacts with users on Slack and handles their requests using available tools.""",
    model="gpt-4o",
    tools=slack_openai_tools,
)

triage_agent = Agent(
    name="Manager Agent",
    instructions="""You are a manager agent that triages tasks to other agents based on their expertise.""",                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           
    model="gpt-4o",
    handoffs=[banking_agent, slack_agent],
)

mas = MultiAgentSystem(
    mas=triage_agent
)

# create a pipeline
pipeline = Pipeline(
    mas=mas
)

In [8]:
# run the benchmark
results = await benchmark_suite_without_injections(
    agent_pipeline=pipeline,
    suite=banking_task_suite,
)

In [9]:
results

{'user_task_0': False,
 'user_task_1': True,
 'user_task_2': True,
 'user_task_3': True,
 'user_task_4': False,
 'user_task_5': True,
 'user_task_6': True,
 'user_task_7': True,
 'user_task_8': True,
 'user_task_9': True,
 'user_task_10': True,
 'user_task_11': False,
 'user_task_12': False,
 'user_task_13': False,
 'user_task_14': True,
 'user_task_15': True}